Patterns And Matching
---

Patterns are a special syntax in Rust for matching against the structure of  
types, both complex and simple. Using patterns in conjunction with match  
expressions and other constructs gives you more control over a program’s  
control flow. A pattern consists of some combination of the following:

- Literals
- Destructured arrays, enums, structs, or tuples
- Variables
- Wildcards
- Placeholders

Some example patterns include x, (a, 3), and Some(Color::Red). In the contexts  
in which patterns are valid, these components describe the shape of data. Our  
program then matches values against the patterns to determine whether it has  
the correct shape of data to continue running a particular piece of code.

To use a pattern, we compare it to some value. If the pattern matches the value,  
we use the value parts in our code. Recall the match expressions in Chapter 6  
that used patterns, such as the coin-sorting machine example. If the value fits  
the shape of the pattern, we can use the named pieces. If it doesn’t, the code  
associated with the pattern won’t run.

This chapter is a reference on all things related to patterns. We’ll cover the  
valid places to use patterns, the difference between refutable and irrefutable  
patterns, and the different kinds of pattern syntax that you might see. By the  
end of the chapter, you’ll know how to use patterns to express many concepts in  
a clear way.

All the Places Patterns Can Be Used
---

Patterns pop up in a number of places in Rust, and you’ve been using them a lot  
without realizing it! This section discusses all the places where patterns are  
valid.

### `match` Arms

As discussed in Chapter 6, we use patterns in the arms of match expressions.  
Formally, match expressions are defined as :  

- the `keyword` match,  
- a `value` to match on, and  
- one or more `match arms` that consist of a pattern and an expression to run if  
the value matches that arm’s pattern, 

like this:

```rust
match VALUE {
    PATTERN => EXPRESSION,
    PATTERN => EXPRESSION,
    PATTERN => EXPRESSION,
}
```

For example, here's the match expression from Listing 6-5 that matches on an  
`Option<i32>` value in the variable `x` :

```rust
match x {
    None => None,
    Some(i) => Some(i + 1),
}
```


The patterns in this match expression are the `None` and `Some(i)` on the left  
of each arrow.

One requirement for match expressions is that they need to be `exhaustive` in  
the sense that all possibilities for the value in the match expression must be  
accounted for. 

One way to ensure you’ve covered every possibility is to have a catchall pattern  
for the last arm: for example, a variable name matching any value can never fail  
and thus covers every remaining case.

The particular pattern _ will match anything, but it never binds to a variable,  
so it’s often used in the last match arm. The _ pattern can be useful when you  
want to ignore any value not specified, for example. We’ll cover the _ pattern  
in more detail in the “Ignoring Values in a Pattern” section later in this  
chapter.

### Conditional `if let` Expressions

In Chapter 6 we discussed how to use if let expressions mainly as a shorter way  
to write the equivalent of a match that only matches one case. Optionally,  
`if let` can have a corresponding else containing code to run if the pattern in  
the if let doesn’t match.

Listing 18-1 shows that it’s also possible to mix and match : 

- `if let`,  
- `else if`, and  
- `else if let` expressions. 

Doing so gives us more flexibility than a match expression in which we can  
express only one value to compare with the patterns. Also, Rust doesn't require  
that the conditions in a series of if let, else if, else if let arms relate to  
each other.

The code in Listing 18-1 determines what color to make your background based on  
a series of checks for several conditions. For this example, we’ve created  
variables with hardcoded values that a real program might receive from user  
input.

> Filename: src/main.rs

In [5]:
{

let favorite_color : Option<&str> = None;
let is_tuesday = false;
let age: Result<u8, _> = "23".parse();

if let Some(color) = favorite_color {
    println!("Using your favorite color, {}, as the background", color);
} else if is_tuesday {
    println!("Tuesday is a greend day");
} else if let Ok(age) = age {
    if age > 30 {
        println!("Using purple as the background color");
    } else {
        println!("Using orange as the background color");
    }
} else {
    println!("Using blue as the background color");
}

}

Using orange as the background color


()

> Listing 18-1: Mixing if let, else if, else if let, and else

If the user specifies a favorite color, that color is used as the background.  
If no favorite color is specified and today is Tuesday, the background color is  
green. Otherwise, if the user specifies their age as a string and we can parse  
it as a number successfully, the color is either purple or orange depending on  
the value of the number. If none of these conditions apply, the background color  
is blue.

This conditional structure lets us support complex requirements. With the  
hardcoded values we have here, this example will print Using purple as the  
background color.

You can see that if let can also introduce shadowed variables in the same way  
that match arms can:  

- the line if let Ok(age) = age introduces a new shadowed age variable that  
contains the value inside the Ok variant.  

This means we need to place the if age > 30 condition within that block:   
- we can’t combine these two conditions into if let Ok(age) = age && age > 30.  

The shadowed age we want to compare to 30 isn’t valid until the new scope starts  
with the curly bracket.

The downside of using if let expressions is that the compiler doesn’t check for  
exhaustiveness, whereas with match expressions it does. If we omitted the last  
else block and therefore missed handling some cases, the compiler would not  
alert us to the possible logic bug.

### `while let` Conditional Loops

Similar in construction to if let, the `while let` conditional loop allows a  
while loop to run for as long as a pattern continues to match. In Listing 18-2  
we code a while let loop that uses a vector as a stack and prints the values in  
the vector in the opposite order in which they were pushed.

In [6]:
let mut stack = Vec::new();

stack.push(1);
stack.push(2);
stack.push(3);

while let Some(top) = stack.pop() {
    println!("{}", top);
}

3
2
1


()

Listing 18-2: Using a while let loop to print values for as long as stack.pop()  
returns Some

This example prints 3, 2, and then 1. The pop method takes the last element out  
of the vector and returns Some(value). If the vector is empty, pop returns None.  
The while loop continues running the code in its block as long as pop returns  
Some. When pop returns None, the loop stops. We can use while let to pop every  
element off our stack.